In [2]:
import tensorflow as tf
import numpy as np
import csv

2022-05-16 19:14:11.686813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 19:14:11.686869: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
path = "../../datasets/translate sentence/result/eng-ind.csv"

In [4]:
with open(path) as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    
    input_data = list()
    target_data = list()
    
    for row in csv_reader:
        input_data.append(row[0])
        target_data.append(row[1])  

In [9]:
def create_model():
    pass